# PoE Harvest Simulation
This notebook simulates harvest plots. The goal is to determine the breakpoint where Crop Rotation is (on average) better non-Crop-Rotation harvest.

Baseline chances have taken from https://forgottenarbiter.github.io/Poe-Harvest-Mechanics/ - both for regular harvest and for crop rotation.

The most interesting part is that the T3 harvest mob chance follows a binomial distribution with p=0.25, which can then be increased through through idols or the atlas passive tree. And that Crop Rotation upgrade chance seems to scale with map tier, which could be especially interesting for T17 harvest farming.



In [1]:
import random
import numpy as np


np.set_printoptions(suppress=True)
sample_amount = 1_000_000

In [2]:

def sample(n, p):

    s1 = random.binomialvariate(n, p)
    s2 = random.binomialvariate(n, p)

    return max(s1, s2)


print(np.mean([sample(3, 0.25) for i in range(sample_amount)]))
print(np.mean([sample(3, 0.25*1.3) for i in range(sample_amount)]))

1.141434
1.408386


The above shows that the average T3 per pair of plots (of which you can only take one because the other one wilts) is 1.14 T3 mobs for T16 harvest without atlas tree investment, and 1.4 T3 mobs if you do specced into it.

In [3]:
def sample_crop_rotation(population = [23, 0, 0, 0], chances = [0.25, 0.2, 0.05, 0], plots=9):

    def upgrade(pop, chances):
        
        upgraded = []
        for i, n in enumerate(pop):
            #if i == len(pop)-1:
            #    continue
            #result[i] = pop[i]
            count = 0
            for _ in range(n):
                if random.uniform(0,1)<chances[i]:
                    count += 1
            upgraded.append(count)
        
        result = []
        for i, n in enumerate(pop):
            if i == 0:
                result.append(pop[i]-upgraded[i])
                continue
            if i == len(pop)-1:
                result.append(pop[i]+upgraded[i-1])
                continue
            result.append(pop[i]-upgraded[i]+upgraded[i-1])

        return result

    #chances = [0.25, 0.2, 0.05, 0]

    #population = [23, 0, 0, 0]

    res = [population]
    for i in range(plots):
        res.append(upgrade(res[-1], chances))

    return res

total = np.array(sample_crop_rotation())

for _ in range(sample_amount-1):
    total = np.add(total, np.array(sample_crop_rotation()))

means = np.divide(total, sample_amount)
print(means)

[[23.        0.        0.        0.      ]
 [17.250176  5.749824  0.        0.      ]
 [12.938635  8.912266  1.149099  0.      ]
 [ 9.702046 10.368255  2.872332  0.057367]
 [ 7.274691 10.722638  4.801632  0.201039]
 [ 5.457264 10.395335  6.706354  0.441047]
 [ 4.092606  9.68018   8.451636  0.775578]
 [ 3.069959  8.766407  9.965851  1.197783]
 [ 2.302701  7.779758 11.222836  1.694705]
 [ 1.727466  6.798853 12.219247  2.254434]]


The above estimates the distribution of T1, T2, T3, T4 mobs after each upgrade step, based on T16 harvest data. 
Interestingly, after only 2 upgrade steps the following plots have at least as many T3 mobs as regular non-specced harvest has per plot pair.
After 5 upgrade steps, the next plot alone has almost as many T3 mobs as a non-crop-rotation but specced harvest has with its maximum amount of 5 plots in total (6.7 vs 7 respectively).

In [ ]:
def cumul(indices, 
          distribution=[[23.,        0.,        0.,        0.      ],
                        [17.250176,  5.749824,  0.,        0.      ],
                        [12.938635,  8.912266,  1.149099,  0.      ],
                        [ 9.702046, 10.368255,  2.872332,  0.057367],
                        [ 7.274691, 10.722638,  4.801632,  0.201039],
                        [ 5.457264, 10.395335,  6.706354,  0.441047],
                        [ 4.092606,  9.68018,   8.451636,  0.775578],
                        [ 3.069959,  8.766407,  9.965851,  1.197783],
                        [ 2.302701,  7.779758, 11.222836,  1.694705],
                        [ 1.727466,  6.798853, 12.219247,  2.254434]],
            ):
    
    
    out = np.zeros(4)
    for i in indices:
        out += distribution[i]
    return out

indices = [0, 1, 2]
print(cumul(indices))

indices = [0, 1, 2, 2]
print(cumul(indices))

indices = [0, 1, 1, 3]
print(cumul(indices))

indices = [0, 0, 2, 2]
print(cumul(indices))

indices = [0, 0, 2, 2, 4]
print(cumul(indices))

indices = [0, 0, 0, 3, 3]
print(cumul(indices))



[53.188811 14.66209   1.149099  0.      ]
[66.127446 23.574356  2.298198  0.      ]
[67.202398 21.867903  2.872332  0.057367]
[79.151961 28.54717   7.09983   0.201039]
[88.404092 20.73651   5.744664  0.114734]


The cumulative 